In [ ]:
import QuantLib as ql

In [ ]:
import csv
import os
from typing import List


def get_project_root():
    '''
    Returns the project root.
    '''
    return os.getcwd()


def read_swap_rates_from_file(date):
    root_dir = get_project_root()
    date_as_int = date.year() * 10000 + date.month() * 100 + date.dayOfMonth()
    file_name = 'swap_rates_' + str(date_as_int) + '.csv'
    file_path = os.path.join(root_dir, 'data/', file_name)
    with open(file_path, 'rt') as file:
        reader = csv.reader(file, delimiter=',')
        rates = [(ql.PeriodParser.parse(str(r[0])), float(r[1])) for r in reader]
    return rates

In [ ]:
def calculate_last_liquid_forward(crv: ql.YieldTermStructureHandle, fsp: ql.Period):
    dc = crv.dayCounter()
    omega = 8.0 / 15.0
    fsp_date = crv.referenceDate() + fsp
    cut_off = crv.timeFromReference(fsp_date)
    llfr_weights = ((ql.Period(25, ql.Years), 1.0), 
                    (ql.Period(30, ql.Years), 0.5), 
                    (ql.Period(40, ql.Years), 0.25), 
                    (ql.Period(50, ql.Years), 0.125))
    llfr = 0.0
    for tenor, weight in llfr_weights:
        time_to_maturity = crv.timeFromReference(fsp_date + tenor)
        llfr += weight * crv.forwardRate(
            cut_off, time_to_maturity, ql.Continuous, ql.NoFrequency, True).rate()
    return llfr


In [ ]:
def bump_quote(quote_handle: ql.RelinkableQuoteHandle, bump = 0.0001):
    quote_handle.linkTo(ql.SimpleQuote(quote_handle.value() + bump))

In [ ]:
# SWAP INDEX CONVENTIONS

ftk_crv_handle = ql.RelinkableYieldTermStructureHandle()

SETTLEMENT_DAYS = 2
BUSINESS_CONVENTION = ql.Unadjusted
DAY_COUNT = ql.SimpleDayCounter()
CALENDAR = ql.NullCalendar()
CCY = ql.EURCurrency()
FXD_FREQUENCY = ql.Annual
FLT_TENOR = ql.Period(6, ql.Months)

ftk_idx = ql.IborIndex("FTK_IDX", FLT_TENOR, SETTLEMENT_DAYS, CCY, CALENDAR,
                       BUSINESS_CONVENTION, False, DAY_COUNT, ftk_crv_handle)

# FTK CURVE CONSTRUCTION

def build_ftk_curve(valuation_date: ql.Date, quote_handles: List[ql.QuoteHandle], idx: ql.IborIndex):
    settlement = CALENDAR.advance(today, SETTLEMENT_DAYS, ql.Days)
    instruments = [ql.SwapRateHelper(q, t, CALENDAR, FXD_FREQUENCY, BUSINESS_CONVENTION, DAY_COUNT, idx) 
                   for t, q in quote_handles]
    crv = ql.PiecewiseLogLinearDiscount(settlement, instruments, DAY_COUNT)
    crv.enableExtrapolation()
    return crv


In [ ]:
# UFR CONVENTIONS

FIRST_SMOOTHING_POINT = ql.Period(20, ql.Years)
ALPHA = 0.1;

ufr_compounded = ql.InterestRate(0.023, DAY_COUNT, ql.Compounded, ql.Annual)
ufr_continuous = ufr_compounded.equivalentRate(ql.Continuous, ql.Annual, 1.0).rate()
ufr_quote_handle = ql.QuoteHandle(ql.SimpleQuote(ufr_continuous))

In [ ]:
llfr_quote_handle = ql.RelinkableQuoteHandle()

def update_llfr_handle():
    llfr = calculate_last_liquid_forward(ftk_crv_handle, FIRST_SMOOTHING_POINT)
    llfr_quote_handle.linkTo(ql.SimpleQuote(llfr))
    print('LLFR observer notified.')
    

observer = ql.Observer(update_llfr_handle)

In [ ]:
# VALUATION DATE

today = CALENDAR.adjust(ql.Date(29, ql.March, 2019))
ql.Settings.instance().evaluationDate = today

In [ ]:

swap_rates = read_swap_rates_from_file(today)
swap_rate_quotes = [(q[0], ql.RelinkableQuoteHandle(ql.SimpleQuote(q[1]))) for q in swap_rates]

# Link swap quotes to observer

for _, quote_handle in swap_rate_quotes:
    observer.registerWith(quote_handle)

In [ ]:
ftk_curve = build_ftk_curve(today, swap_rate_quotes, ftk_idx)
ftk_crv_handle.linkTo(ftk_curve)

In [ ]:
# Link swap quotes to observer

for _, quote_handle in swap_rate_quotes:
    observer.registerWith(quote_handle)

# UFR CURVE CONSTRUCTION
update_llfr_handle()
ufr_curve = ql.UltimateForwardTermStructure(ftk_crv_handle, llfr_quote_handle, ufr_quote_handle, FIRST_SMOOTHING_POINT, ALPHA)

In [ ]:
# print(swap_rate_quotes[0][1].value())
# bump_quote(swap_rate_quotes[15][1], 0.001)
# print(swap_rate_quotes[0][1].value())

# ftk_crv_handle.discount(ftk_crv_handle.referenceDate() + ql.Period(50, ql.Years))
ufr_curve.discount(ftk_crv_handle.referenceDate() + ql.Period(50, ql.Years))